In [83]:
import pandas as pd 
from functools import reduce

# import utils as utils located as src/utils/utils.py
from src.utils import utils
from src.api import data_requests as dr
API_KEY = utils.read_api_key()
import requests
BASE_URL = "https://api.collegefootballdata.com"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}


### Data Collection:
 - Collect NCAA College Football data using APIs like collegefootballdata.com.
 - Install the 'requests' library: pip install requests
 - Example code for retrieving team data:

In [290]:
params = {'searchTerm': 'J.J. McCarthy'}
make_request('player/search', params=params)

[{'id': '4433970',
  'team': 'Michigan',
  'name': 'J.J. McCarthy',
  'firstName': 'J.J.',
  'lastName': 'McCarthy',
  'weight': 196,
  'height': 75,
  'jersey': 9,
  'position': 'QB',
  'hometown': 'La Grange Park, IL',
  'teamColor': '#00274c',
  'teamColorSecondary': '#FFCB05'}]

In [343]:
params = {'year': 2020}
data = make_request('ratings/sp', params=params)
df = pd.DataFrame(data)
offense_df = pd.json_normalize(df['offense']).add_prefix('offense_')
defense_df = pd.json_normalize(df['defense']).add_prefix('defense_')
special_teams_df = pd.json_normalize(df['specialTeams']).add_prefix('specialTeams_')
flattened_df = pd.concat([df.drop(['offense', 'defense', 'specialTeams'], axis=1), offense_df, defense_df, special_teams_df], axis=1)

In [348]:
flattened_df

,year,team,conference,rating,ranking,secondOrderWins,sos,offense_ranking,offense_rating,offense_success,...,defense_success,defense_explosiveness,defense_rushing,defense_passing,defense_standardDowns,defense_passingDowns,defense_havoc.total,defense_havoc.frontSeven,defense_havoc.db,specialTeams_rating
0,2020,Ohio State,Big Ten,28.400000,1.0,None,None,2.0,45.300000,None,...,None,None,None,None,None,None,None,None,None,None
1,2020,Alabama,SEC,28.300000,2.0,None,None,1.0,46.800000,None,...,None,None,None,None,None,None,None,None,None,None
2,2020,Wisconsin,Big Ten,26.500000,3.0,None,None,9.0,38.700000,None,...,None,None,None,None,None,None,None,None,None,None
3,2020,Clemson,ACC,26.100000,4.0,None,None,6.0,42.600000,None,...,None,None,None,None,None,None,None,None,None,None
4,2020,Notre Dame,FBS Independents,21.000000,5.0,None,None,10.0,38.300000,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,2020,Akron,Mid-American,-23.700000,127.0,None,None,130.0,11.400000,None,...,None,None,None,None,None,None,None,None,None,None
127,2020,Bowling Green,Mid-American,-26.200000,128.0,None,None,128.0,12.800000,None,...,None,None,None,None,None,None,None,None,None,None
128,2020,UMass,FBS Independents,-31.300000,129.0,None,None,125.0,16.700000,None,...,None,None,None,None,None,None,None,None,None,None
129,2020,New Mexico State,FBS Independents,-31.500000,130.0,None,None,129.0,12.200000,None,...,None,None,None,None,None,None,None,None,None,None


In [351]:
API_KEY = utils.read_api_key()
import requests
BASE_URL = "https://api.collegefootballdata.com"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

def make_request(endpoint, params=None):
    headers = {'Authorization': f'Bearer {API_KEY}'}
    response = requests.get(f'{BASE_URL}/{endpoint}', headers=headers, params=params)
    response.raise_for_status()
    return response.json() 
    

# RATINGS 
# _________________________________________________________________

def get_sp_ratings(year=None, team=None):
    """
    Fetches historical SP+ ratings for a given year and week.

    Args:
        year (int, optional): The year of the season.
        week (int, optional): The week of the season.

    Returns:
        pd.DataFrame: A DataFrame containing historical SP+ ratings.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request('ratings/sp', params=params)
    df = pd.DataFrame(data)
    offense_df = pd.json_normalize(df['offense']).add_prefix('offense_')
    defense_df = pd.json_normalize(df['defense']).add_prefix('defense_')
    special_teams_df = pd.json_normalize(df['specialTeams']).add_prefix('specialTeams_')
    threshold = int(0.95 * len(df))
    flattened_df = pd.concat([df.drop(['offense', 'defense', 'specialTeams'], axis=1), offense_df, defense_df, special_teams_df], axis=1).dropna(thresh=threshold, axis=1)
    return flattened_df


def get_srs_ratings(year=None, week=None):
    """
    Fetches historical SRS ratings for a given year and week.

    Args:
        year (int, optional): The year of the season.
        week (int, optional): The week of the season.

    Returns:
        pd.DataFrame: A DataFrame containing historical SRS ratings.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request('ratings/srs', params=params)
    return pd.DataFrame(data)


def get_sp_conference_ratings(year=None, conference=None):
    """
    Fetches historical SP+ ratings by conference for a given year and conference.

    Args:
        year (int, optional): The year of the season.
        conference (str, optional): The conference to retrieve SP+ ratings for.

    Returns:
        pd.DataFrame: A DataFrame containing historical SP+ ratings by conference.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request('ratings/sp/conferences', params=params)
    return pd.DataFrame(data)


def get_elo_ratings(year=None, week=None):
    """
    Fetches historical Elo ratings for a given year and week.

    Args:
        year (int, optional): The year of the season.
        week (int, optional): The week of the season.

    Returns:
        pd.DataFrame: A DataFrame containing historical Elo ratings.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request('ratings/elo', params=params)
    return pd.DataFrame(data)

def get_predicted_points(year=None, week=None):
    """
    Fetches predicted points (i.e. expected points or EP) for a given year and week.

    Args:
        year (int, optional): The year of the season.
        week (int, optional): The week of the season.

    Returns:
        pd.DataFrame: A DataFrame containing predicted points data.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request('ppa/predicted', params=params)
    return pd.DataFrame(data)

def get_ppa_teams(year=None):
    """
    Fetches predicted points added (PPA/EPA) data by team for a given year.

    Args:
        year (int, optional): The year of the season.

    Returns:
        pd.DataFrame: A DataFrame containing PPA/EPA data by team.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request('ppa/teams', params=params)
    return pd.DataFrame(data)


def get_ppa_games(year=None):
    """
    Fetches team predicted points added (PPA/EPA) by game for a given year.

    Args:
        year (int, optional): The year of the season.

    Returns:
        pd.DataFrame: A DataFrame containing team PPA/EPA data by game.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request('ppa/games', params=params)
    return pd.DataFrame(data)


def get_ppa_player_games(year=None):
    """
    Fetches player predicted points added (PPA/EPA) broken down by game for a given year.

    Args:
        year (int, optional): The year of the season.

    Returns:
        pd.DataFrame: A DataFrame containing player PPA/EPA data broken down by game.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request('ppa/players/games', params=params)
    return pd.DataFrame(data)


def get_ppa_player_season(year=None):
    """
    Fetches player predicted points added (PPA/EPA) broken down by season for a given year.

    Args:
        year (int, optional): The year of the season.

    Returns:
        pd.DataFrame: A DataFrame containing player PPA/EPA data broken down by season.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request('ppa/players/season', params=params)
    return pd.DataFrame(data)


def get_win_probability(year=None):
    """
    Fetches win probability chart data for a given year.

    Args:
        year (int, optional): The year of the season.

    Returns:
        pd.DataFrame: A DataFrame containing win probability chart data.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request('metrics/wp', params=params)
    return pd.DataFrame(data)


def get_pregame_win_probability(year=None):
    """
    Fetches pregame win probability data for a given year.

    Args:
        year (int, optional): The year of the season.

    Returns:
        pd.DataFrame: A DataFrame containing pregame win probability data.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request('metrics/wp/pregame', params=params)
    return pd.DataFrame(data)


team='Michigan'
# get_play_stats(2019, team=team, week=10)
get_sp_ratings(year = 2021)


,year,team,conference,rating,ranking,offense_ranking,offense_rating,defense_ranking,defense_rating
0,2021,Georgia,SEC,32.000000,1.0,2.0,41.400000,1.0,10.300000
1,2021,Ohio State,Big Ten,29.300000,2.0,1.0,47.200000,20.0,20.600000
2,2021,Alabama,SEC,26.000000,3.0,4.0,40.900000,9.0,15.600000
3,2021,Michigan,Big Ten,23.400000,4.0,19.0,36.500000,10.0,15.800000
4,2021,Cincinnati,American Athletic,21.200000,5.0,13.0,37.500000,11.0,15.900000
...,...,...,...,...,...,...,...,...,...
126,2021,Akron,Mid-American,-23.100000,127.0,113.0,20.300000,128.0,41.700000
127,2021,New Mexico State,FBS Independents,-24.200000,128.0,111.0,20.600000,129.0,43.700000
128,2021,Connecticut,FBS Independents,-25.600000,129.0,129.0,12.700000,119.0,36.800000
129,2021,UMass,FBS Independents,-29.300000,130.0,126.0,16.100000,130.0,43.900000


In [305]:
get_transfer_portal_by_season(2022, team=team)

,season,firstName,lastName,position,origin,destination,transferDate,rating,stars,eligibility
0,2022,CamRon,Jackson,DL,Memphis,Memphis,2022-01-19T12:42:00.000Z,None,3.0,Immediate
1,2022,Malachi,Ledbetter,EDGE,Simon Fraser,Washington State,2022-01-10T06:00:00.000Z,None,NaN,Immediate
2,2022,Isaiah,Johnson,CB,Dartmouth,Syracuse,2022-02-07T16:48:00.000Z,None,NaN,Immediate
3,2022,Maurice,White,DL,Memphis,Central Michigan,2022-08-08T13:42:00.000Z,0.78,2.0,Immediate
4,2022,Emeka,Megwa,RB,Washington,None,2022-08-02T14:06:00.000Z,0.85,3.0,Immediate
...,...,...,...,...,...,...,...,...,...,...
2269,2022,Rourke,Freeburg,LB,Arizona,Toledo,2021-12-15T22:50:00.000Z,None,2.0,Immediate
2270,2022,Khris,Bogle,EDGE,Florida,Michigan State,2021-12-01T21:26:00.000Z,0.9,4.0,Immediate
2271,2022,Daniel,Parker,TE,Missouri,Oklahoma,2021-11-29T14:03:00.000Z,0.86,3.0,Immediate
2272,2022,Landon,Morris,WR,Syracuse,Utah,2021-12-13T18:04:00.000Z,0.86,3.0,Immediate


In [327]:
params={'year': 2022, 'week':1}


,provider,spread,formattedSpread,spreadOpen,overUnder,overUnderOpen,homeMoneyline,awayMoneyline,season,week,homeTeam,awayTeam,homeScore,awayScore
0,Bovada,-32.5,Western Kentucky -32.5,-27.5,73.0,64.5,NaN,NaN,2022,1,Western Kentucky,Austin Peay,38,27
1,William Hill (New Jersey),-32.5,Western Kentucky -32.5,None,72,None,NaN,NaN,2022,1,Western Kentucky,Austin Peay,38,27
2,consensus,-32.5,Western Kentucky -32.5,None,72,None,NaN,NaN,2022,1,Western Kentucky,Austin Peay,38,27
3,Bovada,-10.5,Eastern Michigan -10.5,-13.0,58.0,58.0,-435.0,325.0,2022,1,Eastern Michigan,Eastern Kentucky,42,34
4,William Hill (New Jersey),-11.5,Eastern Michigan -11.5,None,57.5,None,NaN,NaN,2022,1,Eastern Michigan,Eastern Kentucky,42,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,Bovada,-13.5,South Alabama -13.5,-13.5,53.0,53.0,-575.0,400.0,2022,1,South Alabama,Nicholls,48,7
392,William Hill (New Jersey),-13.5,South Alabama -13.5,None,53,None,NaN,NaN,2022,1,South Alabama,Nicholls,48,7
393,consensus,-13.5,South Alabama -13.5,None,53,None,NaN,NaN,2022,1,South Alabama,Nicholls,48,7
394,William Hill (New Jersey),-33.5,Montana -33.5,None,47.5,None,NaN,NaN,2022,1,Montana,Northwestern State,47,0


In [313]:
data = make_request('rankings', params=params)



,rank,school,conference,firstPlaceVotes,points
0,1,Alabama,SEC,54,1566
1,2,Ohio State,Big Ten,6,1506
2,3,Georgia,SEC,3,1455
3,4,Clemson,ACC,0,1292
4,5,Notre Dame,FBS Independents,0,1242
5,6,Texas A&M,SEC,0,1212
6,7,Utah,Pac-12,0,1209
7,8,Michigan,Big Ten,0,1203
8,9,Oklahoma,Big 12,0,956
9,10,Baylor,Big 12,0,884


In [294]:
usage['usage'].iloc[0]

{'overall': 0.4012,
 'pass': 0.8538,
 'rush': 0.1046,
 'firstDown': 0.3657,
 'secondDown': 0.389,
 'thirdDown': 0.522,
 'standardDowns': 0.298,
 'passingDowns': 0.568}

In [295]:
params = {'year': 2022, 'team':'Michigan'}
data = make_request('player/usage', params=params)

In [ ]:
usage

In [296]:
data

[{'season': 2022,
  'id': '4433970',
  'name': 'J.J. McCarthy',
  'position': 'QB',
  'team': 'Michigan',
  'conference': 'Big Ten',
  'usage': {'overall': 0.4012,
   'pass': 0.8538,
   'rush': 0.1046,
   'firstDown': 0.3657,
   'secondDown': 0.389,
   'thirdDown': 0.522,
   'standardDowns': 0.298,
   'passingDowns': 0.568}},
 {'season': 2022,
  'id': '4429096',
  'name': 'Blake Corum',
  'position': 'RB',
  'team': 'Michigan',
  'conference': 'Big Ten',
  'usage': {'overall': 0.2658,
   'pass': 0.0287,
   'rush': 0.422,
   'firstDown': 0.2799,
   'secondDown': 0.244,
   'thirdDown': 0.258,
   'standardDowns': 0.351,
   'passingDowns': 0.148}},
 {'season': 2022,
  'id': '4431536',
  'name': 'Donovan Edwards',
  'position': 'RB',
  'team': 'Michigan',
  'conference': 'Big Ten',
  'usage': {'overall': 0.1727,
   'pass': 0.0705,
   'rush': 0.2401,
   'firstDown': 0.2032,
   'secondDown': 0.171,
   'thirdDown': 0.108,
   'standardDowns': 0.147,
   'passingDowns': 0.156}},
 {'season': 2022,

In [244]:
df

,name,opponent,conference,points,homeAway,week,fumblesRecovered,rushingTDs,puntReturnYards,puntReturnTDs,...,opponent_rushingAttempts,opponent_rushingYards,opponent_yardsPerPass,opponent_completionAttempts,opponent_netPassingYards,opponent_totalYards,opponent_fourthDownEff,opponent_thirdDownEff,opponent_firstDowns,opponent_homeAway
0,Michigan,Rutgers,Big Ten,52,away,1,0,4,-2,0,...,19,14,5.7,14-29,166,180,1-2,3-13,5,home


In [229]:
data

{'school': 'Michigan',
 'conference': 'Big Ten',
 'homeAway': 'away',
 'points': 52,
 'stats': [{'category': 'fumblesRecovered', 'stat': '0'},
  {'category': 'rushingTDs', 'stat': '4'},
  {'category': 'puntReturnYards', 'stat': '-2'},
  {'category': 'puntReturnTDs', 'stat': '0'},
  {'category': 'puntReturns', 'stat': '1'},
  {'category': 'passingTDs', 'stat': '2'},
  {'category': 'kickReturnYards', 'stat': '27'},
  {'category': 'kickReturnTDs', 'stat': '0'},
  {'category': 'kickReturns', 'stat': '2'},
  {'category': 'kickingPoints', 'stat': '10'},
  {'category': 'interceptionYards', 'stat': '81'},
  {'category': 'interceptionTDs', 'stat': '1'},
  {'category': 'passesIntercepted', 'stat': '3'},
  {'category': 'tacklesForLoss', 'stat': '7'},
  {'category': 'defensiveTDs', 'stat': '1'},
  {'category': 'tackles', 'stat': '25'},
  {'category': 'sacks', 'stat': '3'},
  {'category': 'qbHurries', 'stat': '1'},
  {'category': 'passesDeflected', 'stat': '4'},
  {'category': 'possessionTime', 'st

In [228]:
stats_df

,fumblesRecovered,rushingTDs,puntReturnYards,puntReturnTDs,puntReturns,passingTDs,kickReturnYards,kickReturnTDs,kickReturns,kickingPoints,...,yardsPerRushAttempt,rushingAttempts,rushingYards,yardsPerPass,completionAttempts,netPassingYards,totalYards,fourthDownEff,thirdDownEff,firstDowns
0,0,4,-2,0,1,2,27,0,2,10,...,5.3,53,282,5.6,13-27,151,433,3-3,7-17,22


In [226]:
df

,fumblesRecovered,rushingTDs,puntReturnYards,puntReturnTDs,puntReturns,passingTDs,kickReturnYards,kickReturnTDs,kickReturns,kickingPoints,...,yardsPerRushAttempt,rushingAttempts,rushingYards,yardsPerPass,completionAttempts,netPassingYards,totalYards,fourthDownEff,thirdDownEff,firstDowns
0,0,4,-2,0,1,2,27,0,2,10,...,5.3,53,282,5.6,13-27,151,433,3-3,7-17,22


In [224]:
pd.DataFrame(stats).pivot(columns='category', values='stat').reset_index(drop=True)

category,completionAttempts,defensiveTDs,firstDowns,fourthDownEff,fumblesLost,fumblesRecovered,interceptionTDs,interceptionYards,interceptions,kickReturnTDs,...,rushingYards,sacks,tackles,tacklesForLoss,thirdDownEff,totalPenaltiesYards,totalYards,turnovers,yardsPerPass,yardsPerRushAttempt
0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
data = [x for x in data[0]['teams'] if x['school'] == team][0]

In [218]:
data

{'school': 'Michigan',
 'conference': 'Big Ten',
 'homeAway': 'away',
 'points': 52,
 'stats': [{'category': 'fumblesRecovered', 'stat': '0'},
  {'category': 'rushingTDs', 'stat': '4'},
  {'category': 'puntReturnYards', 'stat': '-2'},
  {'category': 'puntReturnTDs', 'stat': '0'},
  {'category': 'puntReturns', 'stat': '1'},
  {'category': 'passingTDs', 'stat': '2'},
  {'category': 'kickReturnYards', 'stat': '27'},
  {'category': 'kickReturnTDs', 'stat': '0'},
  {'category': 'kickReturns', 'stat': '2'},
  {'category': 'kickingPoints', 'stat': '10'},
  {'category': 'interceptionYards', 'stat': '81'},
  {'category': 'interceptionTDs', 'stat': '1'},
  {'category': 'passesIntercepted', 'stat': '3'},
  {'category': 'tacklesForLoss', 'stat': '7'},
  {'category': 'defensiveTDs', 'stat': '1'},
  {'category': 'tackles', 'stat': '25'},
  {'category': 'sacks', 'stat': '3'},
  {'category': 'qbHurries', 'stat': '1'},
  {'category': 'passesDeflected', 'stat': '4'},
  {'category': 'possessionTime', 'st

In [213]:
data[0]['teams']

[{'school': 'Rutgers',
  'conference': 'Big Ten',
  'homeAway': 'home',
  'points': 17,
  'stats': [{'category': 'fumblesRecovered', 'stat': '0'},
   {'category': 'rushingTDs', 'stat': '0'},
   {'category': 'puntReturnYards', 'stat': '35'},
   {'category': 'puntReturnTDs', 'stat': '1'},
   {'category': 'puntReturns', 'stat': '2'},
   {'category': 'passingTDs', 'stat': '1'},
   {'category': 'kickReturnYards', 'stat': '36'},
   {'category': 'kickReturnTDs', 'stat': '0'},
   {'category': 'kickReturns', 'stat': '2'},
   {'category': 'kickingPoints', 'stat': '5'},
   {'category': 'tacklesForLoss', 'stat': '4'},
   {'category': 'defensiveTDs', 'stat': '1'},
   {'category': 'tackles', 'stat': '35'},
   {'category': 'sacks', 'stat': '1'},
   {'category': 'qbHurries', 'stat': '1'},
   {'category': 'passesDeflected', 'stat': '7'},
   {'category': 'possessionTime', 'stat': '22:19'},
   {'category': 'interceptions', 'stat': '3'},
   {'category': 'fumblesLost', 'stat': '0'},
   {'category': 'turnov

In [196]:
def get_player_game_stats(year, week=None, seasonType=None, team=None):
    """
    Fetches player game stats.

    Args:
        year (int): The year of the season.
        season (str): The season of the year, e.g. 'regular' or 'postseason'.
        team (str): The team code of the team, e.g. 'Michigan' for Michigan.

    Returns:
        pd.DataFrame: A DataFrame containing player game stats.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = dr.make_request('games/players', params=params)[0]
    data = [x for x in data['teams'] if x['school'] == team][0]['categories']

    def process_player_stats_category(data): 
        category_name = data['name']
        dfl = []
        for cat_subset in data['types']: 
            subset_name = cat_subset['name']
            category_rename_str = '{}_{}'.format(category_name, subset_name).lower()
            df = pd.DataFrame(cat_subset['athletes'])
            df = df.rename(columns = {'name': 'player_name', 'stat': category_rename_str})
            dfl.append(df) 
        dfc = reduce(lambda left, right: pd.merge(left, right, on=['id', 'player_name'], how='outer'), dfl)
        return dfc
    cat_dfl = [process_player_stats_category(x) for x in data]
    df = pd.concat(cat_dfl)
    return df
get_player_game_stats(2022, team='Michigan', week=11)

,id,player_name,punting_long,punting_in 20,punting_tb,punting_avg,punting_yds,punting_no,kicking_pts,kicking_xp,...,rushing_td,rushing_avg,rushing_yds,rushing_car,passing_qbr,passing_int,passing_td,passing_avg,passing_yds,passing_c/att
0,4258197,Brad Robbins,45,1,0,39.7,119,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4372066,Jake Moody,NaN,NaN,NaN,NaN,NaN,NaN,10,4/4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4429053,A.J. Henning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4429053,A.J. Henning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4587701,Andrel Anthony Jr.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,160900,Will Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4878687,Joe Taylor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4572036,DJ Turner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4432661,Rayshaun Benny,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4429913,Quinten Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
params = {'year': 2019, 'week': 1, 'seasonType': 'regular', 'team': 'Michigan'}
data = dr.make_request('games/players', params=params)[0]
data = [x for x in data['teams'] if x['school'] == team][0]['categories']

In [163]:
[x['name'] for x in data]

['punting',
 'kicking',
 'puntReturns',
 'kickReturns',
 'interceptions',
 'defensive',
 'fumbles',
 'receiving',
 'rushing',
 'passing']

In [164]:
data[0]['types']

[{'name': 'YDS',
  'athletes': [{'id': '4046533', 'name': 'Will Hart', 'stat': '208'}]},
 {'name': 'NO',
  'athletes': [{'id': '4046533', 'name': 'Will Hart', 'stat': '5'}]},
 {'name': 'LONG',
  'athletes': [{'id': '4046533', 'name': 'Will Hart', 'stat': '51'}]},
 {'name': 'In 20',
  'athletes': [{'id': '4046533', 'name': 'Will Hart', 'stat': '0'}]},
 {'name': 'TB',
  'athletes': [{'id': '4046533', 'name': 'Will Hart', 'stat': '0'}]},
 {'name': 'AVG',
  'athletes': [{'id': '4046533', 'name': 'Will Hart', 'stat': '41.6'}]}]

In [175]:
ind = 7
print(data[ind]['name'])
print(data[ind]['types'][0]['name'])
pd.DataFrame(data[ind]['types'][0]['athletes'])


receiving
LONG


,id,name,stat
0,4258191,Dylan McCaffrey,1
1,4372073,Christian Turner,2
2,4046520,Nick Eubanks,5
3,4426385,Zach Charbonnet,5
4,4372063,Ronnie Bell,12
5,4426479,Cornelius Johnson,10
6,4036275,Sean McKeon,28
7,4258173,Nico Collins,28
8,4258170,Tarik Black,36


In [ ]:
for cat_subset in data[0]['types']: 
    subset_name = cat_subset['name']
    category_rename_str = '{}-{}'.format(category_name, subset_name).lower()
    df = pd.DataFrame(cat_subset['athletes'])
    df = df.rename(columns = {'name': 'player_name', 'stat': category_rename_str})
    return df 

In [ ]:

    keys = {
        'punting': ['YDS', 'NO', 'LONG', 'In 20', 'TB', 'AVG'],
        'kicking': ['PTS', 'XP', 'LONG', 'PCT', 'FG'],
        'puntReturns': ['TD', 'LONG', 'AVG', 'YDS', 'NO'],
        'kickReturns': ['TD', 'LONG', 'AVG', 'YDS', 'NO'],
        'interceptions': ['TD', 'YDS', 'INT'],
        'defensive': ['TD', 'QB HUR', 'PD', 'TFL', 'SACKS', 'SOLO', 'TOT'],
        'fumbles':['REC', 'LOST', 'FUM'],
        'receiving': ['LONG', 'TD', 'AVG', 'YDS', 'REC'],
        'rushing': ['LONG', 'TD', 'AVG', 'YDS', 'CAR'],
        'passing': ['QBR', 'INT', 'TD', 'AVG', 'YDS', 'C/ATT']
    }

,punting_yds,punting_no,punting_long,punting_in 20,punting_tb,punting_avg,kicking_pts,kicking_xp,kicking_long,kicking_pct,...,rushing_yds,rushing_car,id,player_name,passing_qbr,passing_int,passing_td,passing_avg,passing_yds,passing_c/att


,id,player_name,punting_yds,punting_no,punting_long,punting_in 20,punting_tb,punting_avg,kicking_pts,kicking_xp,...,rushing_td,rushing_avg,rushing_yds,rushing_car,passing_qbr,passing_int,passing_td,passing_avg,passing_yds,passing_c/att
0,4046533,Will Hart,208,5,51,0,0,41.6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4046532,Quinn Nordin,NaN,NaN,NaN,NaN,NaN,NaN,4,4/4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4372066,Jake Moody,NaN,NaN,NaN,NaN,NaN,NaN,6,0/0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4046538,Lavert Hill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4372063,Ronnie Bell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4427569,Giles Jackson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4258209,Ambry Thomas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4258168,Jordan Anthony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4036258,Carlo Kemp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4258209,Ambry Thomas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
team = 'Michigan'
year = '2019'


In [106]:
def make_request(endpoint, params=None):
    headers = {'Authorization': f'Bearer {API_KEY}'}
    response = requests.get(f'{BASE_URL}/{endpoint}', headers=headers, params=params)
    response.raise_for_status()
    return response.json() 

In [107]:
headers = {'Authorization': f'Bearer {API_KEY}'}
year = '2019'
team = 'Michigan'
params={'year': year, 'team': team}
endpoint='/stats/season'
response = requests.get(f'{BASE_URL}/{endpoint}', headers=headers, params=params)
response.raise_for_status()

In [110]:
response.text

'<!-- HTML for static distribution bundle build -->\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="UTF-8">\n    <title>Swagger UI</title>\n    <link rel="stylesheet" type="text/css" href="./swagger-ui.css" >\n    <link rel="icon" type="image/png" href="./favicon-32x32.png" sizes="32x32" />\n    <link rel="icon" type="image/png" href="./favicon-16x16.png" sizes="16x16" />\n    <style>\n      html\n      {\n        box-sizing: border-box;\n        overflow: -moz-scrollbars-vertical;\n        overflow-y: scroll;\n      }\n\n      *,\n      *:before,\n      *:after\n      {\n        box-sizing: inherit;\n      }\n\n      body\n      {\n        margin:0;\n        background: #fafafa;\n      }\n    </style>\n  </head>\n\n  <body>\n    <div id="swagger-ui"></div>\n\n    <script src="./swagger-ui-bundle.js"> </script>\n    <script src="./swagger-ui-standalone-preset.js"> </script>\n    <script>\n    window.onload = function() {\n      // Begin Swagger UI call region\n      co

In [114]:
BASE_URL = "https://api.collegefootballdata.com"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
API_KEY = utils.read_api_key()

def get_team_season_stats(year, team, seasonType=None, week=None):
    """
    Fetches team statistics by season.

    Args:
        year (int): The year of the season.

    Returns:
        pd.DataFrame: A DataFrame containing team statistics by season.
    """
    params = {k: v for k, v in locals().items() if v is not None}
    data = make_request(f'stats/season', params)
    return pd.DataFrame(data)
get_team_season_stats(team=team, year=year)


,season,team,conference,statName,statValue
0,2019,Michigan,Big Ten,interceptionYards,31
1,2019,Michigan,Big Ten,passAttempts,410
2,2019,Michigan,Big Ten,totalYards,5220
3,2019,Michigan,Big Ten,thirdDownConversions,68
4,2019,Michigan,Big Ten,interceptions,9
5,2019,Michigan,Big Ten,passingTDs,25
6,2019,Michigan,Big Ten,puntReturnYards,271
7,2019,Michigan,Big Ten,fourthDownConversions,12
8,2019,Michigan,Big Ten,kickReturnTDs,1
9,2019,Michigan,Big Ten,netPassingYards,3261


In [90]:
endpoint = 'plays'
https://api.collegefootballdata.com/stats/season
headers = {'Authorization': f'Bearer {API_KEY}'}
response = requests.get(f'{BASE_URL}/{endpoint}', headers=headers, params=params)
response.raise_for_status()


In [92]:
response.json()

https://api.collegefootballdata.com/stats/season
https://api.collegefootballdata.com/stats/season

[]

In [93]:
get_plays(team='Michigan', year='2022', week=1)



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [87]:
team='Michigan'
year='2020'
week=1
params = {'year': year, 'team':team, 'week': week}
data = dr.make_request('/plays', params=params)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [74]:
data

[]

In [52]:
params = {k: v for k, v in locals().items() if v is not None}
data = dr.make_request('games/players', params=params)[0]

HTTPError: 414 Client Error: Request-URI Too Large for url: https://api.collegefootballdata.com/games/players?__name__=__main__&__doc__=Automatically+created+module+for+IPython+interactive+environment&__builtin__=%3Cmodule+%27builtins%27+%28built-in%29%3E&__builtins__=%3Cmodule+%27builtins%27+%28built-in%29%3E&_ih=&_ih=%23+Access+functions%0Aconferences_df+%3D+get_conferences%28%29%0Avenues_df+%3D+get_venues%28%29%0Acoaches_df+%3D+get_coaches%28%29%0Agames_df+%3D+get_games%28%29%0Ascoreboard_df+%3D+get_scoreboard%28%29%0Ateam_records_df+%3D+get_team_records%28%29%0Aseason_calendar_df+%3D+get_season_calendar%28%29%0Agame_media_df+%3D+get_game_media%28%29%0Agame_weather_df+%3D+get_game_weather%28%29%0Aplayer_game_stats_df+%3D+get_player_game_stats%28%29%0Ateam_game_stats_df+%3D+get_team_game_stats%28%29%0Aadvanced_box_scores_df+%3D+get_advanced_box_scores%28%29%0Adrives_df+%3D+get_drives%28%29%0Aplays_df+%3D+get_plays%28%29%0Aplay_types_df+%3D+get_play_types%28%29%0Aplay_stats_df+%3D+get

In [ ]:
[x]

In [58]:
get_player_game_stats(2022, week=None, seasonType=None, team='Michigan')

,school,conference,homeAway,points,categories
0,Michigan,Big Ten,home,34,"[{'name': 'punting', 'types': [{'name': 'LONG'..."


In [ ]:
# from src.api import data_requests as dr

# Accessing the functions
df_conferences = dr.get_conferences()
df_venues = dr.get_venues()
df_coaches = dr.get_coaches()

In [ ]:
# NOT WORKING FUNCTIONS
df_games = dr.get_games()
df_scoreboard = dr.get_scoreboard()
df_team_records = dr.get_team_records()


In [11]:
def function(year=None, test=10): 
    params = {k: v for k, v in locals().items() if v is not None}
    return params

In [12]:
function()

{'test': 10}

In [9]:
def get_games(year=None, season=None, team=None):
    """
    Fetches college football games and results.

    Args:
        year (int, optional): The year of the season. Defaults to None.
        season (str, optional): The season to get games for (e.g. 'regular', 'postseason'). Defaults to None.
        team (str, optional): The abbreviation for the team to get games for (e.g. 'Michigan', 'UGA'). Defaults to None.

    Returns:
        pd.DataFrame: A DataFrame containing game results.
    """
    params = {}
    if year is not None:
        params['year'] = year
    if season is not None:
        params['seasonType'] = season
    if team is not None:
        params['team'] = team
    data = dr.make_request('games', params=params)
    return pd.DataFrame(data)

In [10]:
get_games(year=2022, season=None, team=None)

,id,season,week,season_type,start_date,start_time_tbd,completed,neutral_site,conference_game,attendance,...,away_conference,away_division,away_points,away_line_scores,away_post_win_prob,away_pregame_elo,away_postgame_elo,excitement_index,highlights,notes
0,401426532,2022,1,regular,2022-08-27T16:00:00.000Z,False,True,False,False,None,...,Atlantic Sun,fcs,27.0,"[10, 7, 3, 7]",0.04572357337737354,NaN,NaN,2.6135509766,None,None
1,401405059,2022,1,regular,2022-08-27T16:30:00.000Z,False,True,True,True,None,...,Big Ten,fbs,28.0,"[7, 7, 14, 0]",0.45245654026480564,1636.0,1616.0,8.3493556676,None,None
2,401439512,2022,1,regular,2022-08-27T19:00:00.000Z,False,True,True,False,None,...,SIAC,ii,34.0,"[0, 0, 21, 13]",None,NaN,NaN,None,None,None
3,401413256,2022,1,regular,2022-08-27T19:30:00.000Z,False,True,False,False,None,...,Big Sky,fcs,21.0,"[7, 0, 7, 7]",None,NaN,NaN,None,None,None
4,401413213,2022,1,regular,2022-08-27T19:30:00.000Z,False,True,True,False,None,...,Western Athletic,fcs,17.0,"[7, 10, 0, 0]",0.0011829169683287288,NaN,NaN,5.2414690984,None,FCS Kickoff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3663,401507552,2022,15,regular,2022-12-10T17:00:00.000Z,False,True,False,False,None,...,American Rivers,iii,NaN,[],None,NaN,NaN,None,None,Division III Championship - Semifinal
3664,401507610,2022,15,regular,2022-12-10T17:00:00.000Z,False,True,False,False,None,...,Gulf South,ii,NaN,[],None,NaN,NaN,None,None,Division II Championship - Semifinal
3665,401404145,2022,15,regular,2022-12-10T20:00:00.000Z,False,True,True,False,None,...,American Athletic,fbs,17.0,"[0, 3, 7, 0, 7, 0]",0.9647607373862503,1462.0,1457.0,7.9335168303,None,None
3666,401507611,2022,15,regular,2022-12-10T20:30:00.000Z,False,True,False,False,None,...,Pennsylvania State Athletic,ii,NaN,[],None,NaN,NaN,None,None,Division II Championship - Semifinal


In [6]:

df_season_calendar = dr.get_season_calendar()
df_game_media = dr.get_game_media()
df_game_weather = dr.get_game_weather()
df_player_game_stats = dr.get_player_game_stats()
df_team_game_stats = dr.get_team_game_stats()
df_advanced_box_scores = dr.get_advanced_box_scores()
df_drives = dr.get_drives()
df_plays = dr.get_plays()
df_play_types = dr.get_play_types()
df_play_stats = dr.get_play_stats()
df_play_stat_types = dr.get_play_stat_types()
df_teams = dr.get_teams()
df_fbs_teams = dr.get_fbs_teams()
df_team_rosters = dr.get_team_rosters()
df_team_talent = dr.get_team_talent()
df_team_matchup = dr.get_team_matchup()
df_player_usage = dr.get_player_usage(2022)
df_team_returning_production = dr.get_team_returning_production(2022)
df_player_stats_by_season = dr.get_player_stats_by_season(2022)
df_transfer_portal_by_season = dr.get_transfer_portal_by_season(2022)
df_rankings = dr.get_rankings()
df_betting_lines = dr.get_betting_lines()
df_player_recruiting = dr.get_player_recruiting()
df_team_recruiting = dr.get_team_recruiting()
df_recruit_position_ratings = dr.get_recruit_position_ratings()
df_sp_ratings = dr.get_sp_ratings()
df_srs_ratings = dr.get_srs_ratings()
df_sp_conference_ratings = dr.get_sp_conference_ratings()
df_elo_ratings = dr.get_elo_ratings()
df_predicted_points = dr.get_predicted_points()
df_ppa_teams = dr.get_ppa_teams()
df_ppa_games = dr.get_ppa_games()
df_ppa_player_games = dr.get_ppa_player_games()
df_ppa_player_season = dr.get_ppa_player_season()
df_win_probability = dr.get_win_probability()
df_pregame_win_probability = dr.get_pregame_win_probability()
df_team_season_stats = dr.get_team_season_stats()
df_advanced_season_stats = dr.get_advanced_season_stats()
df_advanced_game_stats = dr.get_advanced_game_stats()
df_team_stat_categories = dr.get_team_stat_categories()
df_nfl_teams = dr.get_nfl_teams()
df_nfl_positions = dr.get_nfl_positions()
df_nfl_draft_picks = dr.get_nfl_draft_picks()


HTTPError: 400 Client Error: Bad Request for url: https://api.collegefootballdata.com/records

In [5]:

def search_player(query):
    """
    Searches for player information based on the query.

    Args:
        query (str): A search query for a player.

    Returns:
        pd.DataFrame: A DataFrame containing player information.
    """
    data = make_request('player/search', {'search': query})
    return pd.DataFrame(data)
def make_request(endpoint, params=None):
    headers = {'Authorization': f'Bearer {API_KEY}'}
    response = requests.get(f'{BASE_URL}/{endpoint}', headers=headers, params=params)
    response.raise_for_status()
    return response.json()
    
API_KEY = utils.read_api_key()


In [8]:
import requests
BASE_URL = "https://api.collegefootballdata.com"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}



HTTPError: 400 Client Error: Bad Request for url: https://api.collegefootballdata.com/player/search?search=Joe+Burrow

In [25]:
mich_player_stats = get_player_game_stats(2022, week=None, seasonType=None, team='Michigan')

In [26]:
mich_player_stats

,id,teams
0,401405137,"[{'school': 'Nebraska', 'conference': 'Big Ten..."
1,401405067,"[{'school': 'Michigan', 'conference': 'Big Ten..."
2,401404149,"[{'school': 'Michigan', 'conference': 'Big Ten..."
3,401405097,"[{'school': 'Michigan', 'conference': 'Big Ten..."
4,401405125,"[{'school': 'Michigan State', 'conference': 'B..."
5,401405077,"[{'school': 'Michigan', 'conference': 'Big Ten..."
6,401405100,"[{'school': 'Iowa', 'conference': 'Big Ten', '..."
7,401405115,"[{'school': 'Michigan', 'conference': 'Big Ten..."
8,401405108,"[{'school': 'Indiana', 'conference': 'Big Ten'..."
9,401405133,"[{'school': 'Michigan', 'conference': 'Big Ten..."


In [21]:
mich_player_stats.columns

Index(['id', 'season', 'week', 'season_type', 'start_date', 'start_time_tbd',
       'completed', 'neutral_site', 'conference_game', 'attendance',
       'venue_id', 'venue', 'home_id', 'home_team', 'home_conference',
       'home_division', 'home_points', 'home_line_scores',
       'home_post_win_prob', 'home_pregame_elo', 'home_postgame_elo',
       'away_id', 'away_team', 'away_conference', 'away_division',
       'away_points', 'away_line_scores', 'away_post_win_prob',
       'away_pregame_elo', 'away_postgame_elo', 'excitement_index',
       'highlights', 'notes'],
      dtype='object')

In [4]:
conferences_df = dr.get_conferences()
venues_df = dr.get_venues()
coaches_df = dr.get_coaches()
player_df = dr.search_player('JJ McCarthy')
player_usage_df = dr.get_player_usage(2022)
returning_production_df = dr.get_team_returning_production(2022)
player_stats_by_season_df = dr.get_player_stats_by_season(2022)
transfer_portal_df = dr.get_transfer_portal_by_season(2022)


HTTPError: 400 Client Error: Bad Request for url: https://api.collegefootballdata.com/player/search?search=JJ+McCarthy

In [ ]:
import requests

def get_teams():
    url = 'https://api.collegefootballdata.com/teams'
    response = requests.get(url)
    teams = response.json()
    return teams

teams = get_teams()


In [ ]:
gtoydQGp9rsHwyx2oUUfKz4rS/TPGlSFSHTc6NZy2D40Q02TkrMrLgb+0MlQ2tp5

### Preprocessing:
 - Clean and preprocess the collected data.
 - Handle missing values, outliers, and convert categorical data into numerical form.


In [ ]:

# Convert the data into a DataFrame
team_df = pd.DataFrame(teams)

# Drop unnecessary columns
team_df = team_df.drop(columns=['id', 'abbreviation'])

# Encode categorical columns
team_df['conference'] = team_df['conference'].astype('category')
team_df['conference_code'] = team_df['conference'].cat.codes

### Feature Engineering:
 - Create new features from existing data.
 - Normalize or standardize the data if required.


In [ ]:
# Example: Add winning percentage
team_df['winning_percentage'] = team_df['wins'] / (team_df['wins'] + team_df['losses'])

# Standardize numerical features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
team_df[['wins', 'losses', 'winning_percentage']] = scaler.fit_transform(team_df[['wins', 'losses', 'winning_percentage']])


### Model Training:
 - Train a machine learning model to rank teams based on their features.
 - Example: Use a clustering algorithm like KMeans to group similar teams together.


In [ ]:
from sklearn.cluster import KMeans

# Select features for clustering
features = ['conference_code', 'wins', 'losses', 'winning_percentage']
X = team_df[features]

# Train KMeans model
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X)

# Add cluster labels to DataFrame
team_df['cluster'] = kmeans.labels_
